In [1]:
%matplotlib inline

이 챕터는 기계 학습의 일반적인 작업을 위해 API를 통해 실행됩니다. 더 자세히 알아 보려면 각 섹션의 링크를 참조하십시오.

## Working with data
PyTorch는 두 가지 [데이터 작업을 위한 기본 요소](https://pytorch.org/docs/stable/data.html)인 ``torch.utils.data.DataLoader``, ``torch.utils.data.Dataset``가 있습니다. ``torch.utils.data.Dataset``은 사용하고자 하는 데이터 샘플 및 해당 샘플의 레이블을 저장하고, ``torch.utils.data.DataLoader``는 ``torch.utils.data.Dataset``의 레이블을 이용하여 데이터 샘플에 쉽게 접근할 수 있도록 합니다.

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

PyTorch는 [TorchText](https://pytorch.org/text/stable/index.html), [TorchVision](https://pytorch.org/vision/stable/index.html), [TorchAudio](https://pytorch.org/audio/stable/index.html)를 통해 각 도메인별 라이브러리를 제공합니다. 모든 라이브러리는 샘플 데이터가 포함되어 있으며, 이번 튜토리얼에서는 TorchVision 데이터를 사용합니다.

``torchvision.datasets`` 모듈에는 CIFAR, COCO([full list here](https://pytorch.org/docs/stable/torchvision/datasets.html)) 등이 존재하며, 아래 예시에서는 FashionMNIST를 사용합니다. 모든 TorchVision의 ``Dataset``에는 샘플과 레이블을 각각 수정하기 위한 두 가지 인수 ``transform``, ``target_transform``가 포함됩니다.

In [3]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

``Dataset``을 ``DataLoader``에 대한 인수로 전달합니다. 이는 데이터 세트에 대한 편집 가능성, 자동 배치(automatic batching), 샘플링, 셔플링 및 multiprocess 기능을 지원합니다. 예시에서는 배치 크기를 64, 즉 데이터 읽기 영역의 각 요소는 64개의 feature와 레이블로 구성된 배치를 반환합니다.

In [4]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


## Creating models

PyTorch에서 신경망을 정의하기 위해 [nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html)에서 상속하는 클래스를 만들어줍니다. ``__init__`` 함수에서 Layer를 정의하고, ``forward`` 함수에서 데이터가 네트워크를 통과하는 방법을 지정합니다. 신경망에서 작업을 가속화하기 위해 가능한 경우 GPU를 사용합니다.

In [5]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten()
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


## Optimizing the Model Parameters

모델을 학습하기 위해 [loss function](https://pytorch.org/docs/stable/nn.html#loss-functions>)과 [optimizer](https://pytorch.org/docs/stable/optim.html)를 사용합니다.



In [6]:
loss_fn = nn.CrossEntropyLoss()
learning_rate = 1e-3
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

단일 학습 루프에서 모델은 학습 데이터셋에 대한 예측을 수행하고, 예측 오차를 역전파하여 모델의 파라미터를 조정합니다.


In [7]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

학습 중인지 확인하기 위해 테스트 데이터셋에 대한 모델의 성능을 확인할 수 있습니다.

In [8]:
def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

훈련 프로세스는 여러 반복(*epochs*)에 걸쳐 수행됩니다. 각 epoch 동안 모델은 더 나은 예측을 위해 파라미터를 학습하며, 각 epoch에서 모델의 정확성과 loss를 출력할 수 있습니다. 또한, 매 epoch마다 정확도가 증가하고 loss가 감소하는 것을 확인할 수 있습니다.

In [9]:
epochs = 15
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model)
print("Done!")

Epoch 1
-------------------------------
loss: 2.295450  [    0/60000]
loss: 2.293073  [ 6400/60000]
loss: 2.278504  [12800/60000]
loss: 2.282501  [19200/60000]
loss: 2.273211  [25600/60000]
loss: 2.258452  [32000/60000]
loss: 2.248237  [38400/60000]
loss: 2.228594  [44800/60000]
loss: 2.240276  [51200/60000]
loss: 2.221318  [57600/60000]
Test Error: 
 Accuracy: 51.8%, Avg loss: 0.034745 

Epoch 2
-------------------------------
loss: 2.212354  [    0/60000]
loss: 2.207739  [ 6400/60000]
loss: 2.160400  [12800/60000]
loss: 2.176181  [19200/60000]
loss: 2.168270  [25600/60000]
loss: 2.146453  [32000/60000]
loss: 2.119934  [38400/60000]
loss: 2.083791  [44800/60000]
loss: 2.126453  [51200/60000]
loss: 2.077550  [57600/60000]
Test Error: 
 Accuracy: 53.2%, Avg loss: 0.032452 

Epoch 3
-------------------------------
loss: 2.082280  [    0/60000]
loss: 2.068733  [ 6400/60000]
loss: 1.965958  [12800/60000]
loss: 1.997126  [19200/60000]
loss: 2.002057  [25600/60000]
loss: 1.967370  [32000/600

모델이 처음에는 성능이 좋지 않다는 것을 알았을 것입니다. 더 많은 `epochs`에 대해 루프를 실행하거나 `learning_rate`를 더 큰 숫자로 조정해보세요. 또한 우리가 선택한 모델 구성이 이러한 종류의 문제에 대한 최적의 구성이 아닐 수도 있습니다. 이후 과정에서는 Vision 문제에 대해 작동하는 모델 구조에 대해 자세히 알아볼 것입니다.

Saving Models
-------------
모델을 저장하는 일반적인 방법은 internal state dictionary을 직렬화하는 것입니다.

In [10]:
torch.save(model.state_dict(), "data/model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


Loading Models
----------------------------

모델을 불러오는 프로세스에는 모델 구조를 다시 만들고 state를 불러오는 작업이 포함됩니다.

In [11]:
model = NeuralNetwork()
model.load_state_dict(torch.load("data/model.pth"))

<All keys matched successfully>

이제 이 모델을 사용하여 예측할 수 있습니다.

In [12]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
